In [ ]:
import sys
import json
from pathlib import Path

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

# Root path of project relative to this notebook
ROOT = Path('..')

sys.path.insert(1, str(ROOT / 'scripts'))
from datamodels import *
from utils import *

### Read patient data from local file

In [ ]:
df = pd.read_csv(ROOT / 'datasets' / 'drawing_evaluations.csv', index_col=0)

# Convert non-primitive fields and normalize paths
df['processed_path'] = df['processed_path'].apply(lambda x: ROOT / x)
df['image_path'] = df['image_path'].apply(lambda x: ROOT / x)
df['template_path'] = df['template_path'].apply(lambda x: ROOT / x)
df['drawing_box'] = df['drawing_box'].apply(lambda x: Box.load(x))
df['template_box'] = df['template_box'].apply(lambda x: Box.load(x))

# Remove all unnecessary columns from our dataset
feat_keys = ['processed_path']
group_columns = ['diagnosis']
df = df[group_columns + feat_keys].copy()

df.head()

### Load custom MLP model

In [ ]:
from quick_draw_learner import QDClassifier

### Create a file to hold intermediate results

In [ ]:
results_file = ROOT / 'results' / 'qd_ungrouped.csv'

if not results_file.exists():
    pd.DataFrame(columns=[
        'Trial',
        'Random seed', 
        'Parameters', 
        'Accuracy',
        'Precision',
        'Recall',
        'Area under ROC']).to_csv(results_file, index=False)

trial_results = pd.read_csv(results_file).to_dict(orient='records')

### Build and train...

In [ ]:
from bananas.statistics.scoring import ScoringFunction
    
# Define all possible hyperparameters
parameters = {
    'random_seed': [0],
    'scoring_function': [ScoringFunction.ACCURACY],
    'kernel_size': [5, 7, 9, 11],
}

# Parameters that are changed by hand
batch_size = 48
test_split = .2
validation_split = .2

target_label = 'SANO'
target_column = 'diagnosis'

In [ ]:
from bananas.dataset import DataSet, DataType, Feature
from bananas.sampling.cross_validation import DataSplit
from bananas.statistics import scoring
from bananas.statistics.scoring import ScoringFunction
from bananas.hyperparameters.gridsearch import GridSearch

# Perform 3 trials per parameter set to later compute the average
trial_count = 3
score_history = {}

# Execute the independent trials
for trial_num in tqdm(range(trial_count), leave=False, desc='Trial'):

    # Create a single feature containing all image data
    features = [Feature(
        ImageAugmenterLoader(df['processed_path'].values),
        kind=DataType.HIGH_DIMENSIOAL,
        sample_size=4,
        random_seed=0)]

    # Define target feature
    target_feature = Feature(
        (df[target_column] == target_label).values, random_seed=0)

    # Compute the overall class balance
    true_class_balance = sum(target_feature[:] / len(target_feature))


    # Always re-initialize the randopm seed
    random_seed = trial_num

    while True:

        # Change seed at every iteration to ensure different dataset split
        random_seed = np.random.RandomState(seed=random_seed).randint(1E6)

        # Build dataset, making sure that we have a left-out subset
        dataset = DataSet(
            features,
            name=target_label,
            target=target_feature,
            random_seed=random_seed,
            batch_size=batch_size,
            test_split=test_split,
            validation_split=validation_split)

        # Rebuild dataset unless test class balance is within 5% of ground truth
        test_idx = dataset.sampler.subsamplers[DataSplit.VALIDATION].data
        test_classes = target_feature[test_idx]
        test_class_balance = sum(test_classes) / len(test_classes)
        if abs(test_class_balance - true_class_balance) < .05: break

    # Instantiate learner using pre-trained model
    learner = GridSearch(QDClassifier, learner_parameters=parameters, n_jobs=4)

    # Train learner using train dataset
    try:
        learner.train(
            dataset.input_fn,
            progress=True,
            max_steps=1000)
    except KeyboardInterrupt:
        pass

    # Test all learner predictions using left-out validation subset
    for learner_, params in learner.parameters_.items():
        
        X, y = dataset[test_idx]
        y = learner_.label_encoder_.transform(y)
        y_ = learner_.predict_proba(X)
        
        # Add manual parameters
        params = {**params, 'batch_size': batch_size}

        # Store trial results
        trial_results.append({
            'Trial': trial_num,
            'Random seed': random_seed,
            'Parameters': json.dumps(params),
            'Accuracy': scoring.score_accuracy(y, y_),
            'Precision': scoring.score_precision(y, y_),
            'Recall': scoring.score_recall(y, y_),
            'Area under ROC': scoring.score_auroc(y, y_),
            'History': json.dumps(learner.history_[learner_].scores)
        })

    df_results = pd.DataFrame.from_dict(trial_results)
    df_results = df_results.drop_duplicates()
    df_results = df_results.sort_values('Accuracy', ascending=False)
    df_results.to_csv(results_file, index=False)

In [ ]:
df_results.head(15)